<h1  align="center"><b> TRATAMENTO DE DADOS </b></h1>

`Objetivo Geral:` Importar os dados pluviométricos do município de São Paulo e realizar o tratamento dos dados para que possam ser utilizados na análise exploratória dos próximos projetos.

`Dados:` Os dados foram retirados do [Banco de Dados Hidrológicos](http://www.hidrologia.daee.sp.gov.br/) do Portal do Departamento de Águas e Energia Elétrica do Estado de São Paulo (DAEE) no dia 21/12/2023. 

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from matplotlib.pylab import rcParams
rcParams['figure.figsize'] = 15, 6

<h2 align="center"><b> Análise Inicial </b></h2>

`Objetivo da Etapa:` Importar os dados e realizar uma análise inicial para verificar e corrigir os problemas. Abrindo o arquivo podemos perceber diversos problemas:
1. `Linhas de Cabeçalho:` Devemos Remover na Importação (skiprows = 10)
2. `Vírgula como Separador Decimal ` Podemos remover como no Projeto 1, transformando ',' em '.' e converter para float
3. `Dados Faltantes: ` Quando não há dados, o DAEE coloca '---' no lugar, devemos substituir por NaN.
4. `Formato da Data não é Conveniente:` Reorganizar os Dados


<h2 align="center"><b> Primeira Opção </b></h2>
&emsp;&emsp; Vou resolver os problemas 2 e 3 de duas formas diferentes para praticar. Dependendo do problema, uma forma pode ser mais conveniente que a outra. A primeira opção é a seguinte:

In [3]:
tabela_chuva = pd.read_csv('./Chuva Mensal.csv', sep = ';', skiprows = 10, skipfooter = 1, decimal = ',', na_values = '---', engine = 'python') # Lendo a tabela já convertendo para float e substituindo os valores '---' por NaN.

tabela_chuva = tabela_chuva.fillna(tabela_chuva.mean()) # Substituindo os NaN pela média mensal
tabela_chuva = tabela_chuva.drop(tabela_chuva.index[-1]) # Removendo a linha ultimo ano, além de estar com a média em vários meses (Agosto em diante), vamos usá-la para testar o modelo

<h2 align="center"><b> Segunda Opção </b></h2>

In [2]:
tabela_chuva = pd.read_csv('./Chuva Mensal.csv', sep = ';', skiprows = 10)

tabela_chuva = tabela_chuva.apply(lambda x: x.str.replace(',', '.')) # troca virgula por ponto

média = tabela_chuva.iloc[-1] # Separando a média mensal
tabela_chuva = tabela_chuva.drop(tabela_chuva.index[-2:]) # Removendo a linha da média mensal
tabela_chuva = tabela_chuva.apply(lambda x: pd.to_numeric(x.str.replace('---', str(média[x.name])))) # Substituindo os valores '---' pela média mensal e convertendo para float

<h2 align="center"><b> Finalização </b></h2>

`Objetivo da Etapa:` Resolver o problema 4 e salvar os dados em um arquivo csv para serem utilizados na análise exploratória.

In [3]:
tabela_chuva = tabela_chuva.drop(columns = 'Ano')
valores = list(tabela_chuva.values.flatten()) # Transformando a tabela em um array
índice = pd.date_range('1985', periods = len(valores), freq = 'M') # Criando um index com o mesmo tamanho do array e com frequência mensal

série_chuva = pd.Series(valores, index = índice) # Criando a série temporal
série_chuva.to_csv('Série Tratada - Chuva Mensal.csv', sep = ';') # Salvando a série temporal

In [4]:
display(série_chuva)

1985-01-31    184.9
1985-02-28    242.9
1985-03-31    230.0
1985-04-30     59.0
1985-05-31    126.2
              ...  
2022-08-31     35.2
2022-09-30    131.3
2022-10-31    117.3
2022-11-30    197.7
2022-12-31    200.4
Freq: M, Length: 456, dtype: float64

In [ ]:
média_móvel = série_chuva.rolling(window = 3).mean() # Calculando a média móvel com janela de 3 meses

plt.plot(série_chuva, label = 'Série Original')
plt.plot(média_móvel, label = 'Média Móvel', color = 'red')
plt.legend(loc = 'best')
plt.show()

![média_móvel](./Gráficos/média_móvel.png)